prerequisites:  
tenpy: https://tenpy.github.io/INSTALL.html  
tensorflow: https://www.tensorflow.org/install  

In [1]:
from AD_tools import *
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input,Dense,Conv2D,Conv1D,MaxPooling2D,MaxPooling1D,UpSampling2D,UpSampling1D,Flatten, Activation, BatchNormalization, GlobalAveragePooling2D,add

/export/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/export/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/export/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/export/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/f

In [2]:
tf.__version__

'1.13.1'

In [3]:
import tenpy
tenpy.__version__

'0.5.0'

# Load data
This was previously calcualted on a separate session for 130x100 states for the whole phase diagram

In [4]:
Us = np.concatenate([np.linspace(0.,2.,30),np.linspace(2.,8,100)])
Vs = np.linspace(0,5,100)

In [5]:
dat = np.load("../test_data/329/329_pbc_test-2D_L-64_D-100_spectra.npz")
x_test2D = np.concatenate([np.exp(-1*dat[str(key)][:,12]) for key in [5,6,0,1,2,3,4]])
x_test2D = x_test2D.reshape(*x_test2D.shape,1)
x_test2D.shape

FileNotFoundError: [Errno 2] No such file or directory: '../test_data/329/329_pbc_test-2D_L-64_D-100_spectra.npz'

# Define CNN

In [ ]:
def scc_cnn1D(loss,optimizer,activation0,activation,x_shape):
    in_dim = x_shape
    input_img = Input(shape=in_dim)
   
    x1 = Conv1D(64, 3, activation=activation0, padding='same')(input_img) # 100 100 64
    x2 = MaxPooling1D(2, padding='same')(x1) # 50 50 64
    x2 = Conv1D(64, 3, activation=activation0, padding='same')(x2) # 50 50 128
    x3 = MaxPooling1D(2, padding='same')(x2) # 25 25 128
    encoded = Conv1D(64, 3, activation=activation0, padding='same')(x3) # 25 25 128

    y=UpSampling1D(2)(encoded) # 50 50 128
    y=add([x2,y]) # 50 50 128
    y=Conv1D(64, 3, activation=activation0, padding='same')(y) # 50 50 128
    y=UpSampling1D(2)(y) # 100 100 128
    y=add([x1,y]) # 100 100 128
    decoded = Conv1D(x_shape[-1], 3, activation=activation, padding='same')(y) # 100 100 4

    cnn = Model(input_img, decoded)

    cnn.compile(loss=loss,optimizer=optimizer)#,metrics=['accuracy']) #adadelta
    return cnn

In [ ]:
cnn = scc_cnn1D("mse","adam","relu","relu",(64,1,))
cnn.summary()

# Training
Usually compute extra training set from test set, but since test set is big enough we can just use parts of the test set for training

In [ ]:
def pick_Us(Umin,Umax):
    return np.arange(np.where(Us>Umin)[0][0]-1,np.where(Us<Umax)[0][-1]+1,1)    
def pick_Vs(Vmin,Vmax):
    return np.arange(np.where(Vs>Vmin)[0][0]-1,np.where(Vs<Vmax)[0][-1]+1,1)    

In [ ]:
trains0 = [[(0,1.3),(0,1.3)]]
x_train = []
for train in trains0:
    x_train.append(x_test2D.reshape(130,100,*x_test2D.shape[1:])[pick_Us(*train[1])][:,pick_Vs(*train[0])].reshape(-1,*x_test2D.shape[1:]))
x_train = np.concatenate(x_train)
x_train.shape

In [ ]:
def train_loss2(y_true,y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.sum((y_true - y_pred)**2))

In [ ]:
cnn0 = training(x_train, scc_cnn1D, name = "", provide_cnn = False,
             load_prev = False, num_epochs = 10,  verbose_val = 1, batch_size = 24, shuffle = True, early = False,
             loss = train_loss2, activation0 = 'relu', activation = 'tanh', optimizer = "adam")

**Expected Output:**

The expected output should look like this

<img src="Images/Learning_curve2.png">

In [ ]:
out = 100*eval_loss(x_test2D,cnn0.predict(x_test2D),norm=norm2) #in order to get percent
out = out.reshape(130,100)

In [ ]:
x,y = np.meshgrid(Vs,Us)

In [ ]:
import matplotlib.patches as patches
cmap = plt.get_cmap("plasma") #'viridis'
fig,axs = plt.subplots(figsize=(8,5),squeeze=False)

nbins=100

ax = axs[0,0]
levels = MaxNLocator(nbins=nbins).tick_values(out.min(),out.max()) #out.max()
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
im = ax.pcolormesh(x, y, out, cmap=cmap, norm=norm)
cbar = fig.colorbar(im, ax=ax)
cbar.ax.tick_params(labelsize=20)
ax.set_title("Loss [%]",fontsize=20)
ax.set_xlabel("V",fontsize=24)
ax.set_ylabel("U",fontsize=24)
ax.tick_params(labelsize=20)
#ax.set_ylim(*ylim)

**Expected Output:**

This should return the whole phase diagram

<img src="Images/Phase_Diagram.png">

In [ ]:
fig,axs = plt.subplots(ncols=3,figsize=(18,5))
ax = axs[0]
col=6
ax.plot(y[:,col],out[:,col])
ax.set_title("V = {:.3f}".format(x[0,col]))

ax = axs[1]
row = 6
ax.plot(x[row],out[row])
ax.set_title("U = {:.3f}".format(y[row,0]))

ax = axs[2]
row = 80
ax.plot(x[row],out[row])
ax.set_title("U = {:.3f}".format(y[row,0]))

**Expected Output:**

This returns you different cross sections of the Phase Diagram:

<img src="Images/Phase_Boundaries1.png">